# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [41]:
import redis

In [42]:
r = redis.Redis(host='localhost', port=6379)

In [43]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [44]:
# Stockage du message "hello world" dans la clé "greetings"
r.set('greetings', 'hello world')

# Récupération du message stocké dans la clé "greetings"
value = r.get('greetings')

# Affichage du message
print(value)

b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [45]:
# Création d'une nouvelle clé "connections" avec la valeur 0
r.set('connections', 0)

# Incrémentation de la valeur de la clé "connections"
r.incr('connections')

# Récupération de la valeur de la clé "connections"
connections_value = r.get('connections')

# Affichage de la valeur de la clé "connections"
print(connections_value)

# Test de l'incrémentation de la clé "greetings"
try:
    r.incr('greetings')
except redis.RedisError as e:
    print("Erreur lors de l'incrémentation de 'greetings':", e)

b'1'
Erreur lors de l'incrémentation de 'greetings': value is not an integer or out of range


### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [46]:
# Création d'une clé "colors" contenant une liste de couleurs
r.rpush('colors', 'red', 'black', 'blue')



6

**Q** : Display the second element of the list with `LINDEX`

In [47]:
# Affichage du second élément de la liste de couleurs
second_color = r.lindex('colors', 1)

# Affichage du second élément de la liste de couleurs
print(second_color)

b'black'


**Q** : Display the full list with `LRANGE`

In [48]:
# Récupération de la liste de couleurs
colors = r.lrange('colors', 0, -1)

# Affichage de la liste de couleurs
print(colors)

[b'red', b'black', b'blue', b'red', b'black', b'blue']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [49]:
# Création d'une clé "nosql" 
r.sadd('nosql', 'redis', 'mongodb', 'orientdb')

0

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [50]:
# Test si la valeur "mysql" est dans la clé "nosql"
is_mysql_in_set = r.sismember('nosql', 'mysql')

# Affichage du résultat
print(is_mysql_in_set)

0


**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [51]:
# Ajout de la valeur "hbase" à la clé "nosql"
r.sadd('nosql', 'hbase')

# Récupération de la liste des valeurs de la clé "nosql"
nosql = r.smembers('nosql')

# Affichage des résultats
print(nosql)

{b'orientdb', b'mongodb', b'redis', b'hbase'}


In [52]:
# Réajout de la valeur "redis" à la clé "nosql"
add_redis_result = r.sadd('nosql', 'redis')

# Récupération de la liste des valeurs de la clé "nosql"
nosql = r.smembers('nosql')

# Affichage des résultats
print(nosql)

{b'orientdb', b'mongodb', b'redis', b'hbase'}


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [53]:
# Création d'une clé "top14" contenant les équipes du Top 14 et leur score
r.zadd('top14', {
    'Agen': 10,
    'Bordeaux': 33,
    'Brive': 32,
    'Castres': 29,
    'Clermont': 38,
    'Grenoble': 24,
    'La Rochelle': 26,
    'Montpellier': 32,
    'Oyonnax': 14,
    'Pau': 20,
    'Racing': 40,
    'Stade Français': 22,
    'Toulon': 36,
    'Toulouse': 36
})

# Récupération des équipes du Top 14 et de leur score
top14 = r.zrange('top14', 0, -1, withscores=True)

# Affichage des résultats
print(top14)

[(b'Agen', 10.0), (b'Oyonnax', 14.0), (b'Pau', 20.0), (b'Stade Fran\xc3\xa7ais', 22.0), (b'Grenoble', 24.0), (b'La Rochelle', 26.0), (b'Castres', 29.0), (b'Brive', 32.0), (b'Montpellier', 32.0), (b'Bordeaux', 33.0), (b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [54]:
# Récupération du score de Toulon
toulon_score = r.zscore('top14', 'Toulon')

# Récupération du rang de Toulon
toulon_rank = r.zrank('top14', 'Toulon')

# Affichage du score de Toulon
print(f"Score de Toulon : {toulon_score}"), print(f"Rang de Toulon : {toulon_rank}")

Score de Toulon : 36.0
Rang de Toulon : 10


(None, None)

**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [55]:
# Récupération du vrai rang de Toulon
toulon_rank = r.zrevrank('top14', 'Toulon')

print(f"Vrai rang de Toulon : {toulon_rank}")

Vrai rang de Toulon : 3


**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [56]:
# Obtention des 3 meilleures équipes
top_3 = r.zrevrange('top14', 0, 2, withscores=True)

# Obtention des équipes avec plus de 35 points
above_35 = r.zrangebyscore('top14', 35, '+inf', withscores=True)

# Affichage des résultats
print("Les 3 meilleures équipes :", top_3), print("Équipes avec plus de 35 points :", above_35)

Les 3 meilleures équipes : [(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]
Équipes avec plus de 35 points : [(b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


(None, None)

### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [57]:
# Création d'une clé "user:1" contenant les informations d'un utilisateur
r.hmset('user:1', {'id': 1, 'name': 'Jean', 'age': 22})

# Récupération des informations de l'utilisateur
user = r.hgetall('user:1')

# Affichage des informations de l'utilisateur
print(user)

{b'id': b'1', b'name': b'Jean', b'age': b'22', b'city': b'Lyon'}


c:\Users\hugoc\anaconda3\envs\nosql\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  


**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [58]:
# Mise à jour de la ville de l'utilisateur
r.hset('user:1', 'city', 'Lyon')

# Mise à jour de nom de l'utilisateur
r.hset('user:1', 'name', 'Paul')

# Récupération des informations de l'utilisateur
user = r.hgetall('user:1')

# Affichage des informations de l'utilisateur
print(user)

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}


## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [59]:
import json

# Création de la requête
put_response = {"id": 10, "name": "jean"}

# Insertion de la réponse dans le cache
r.set('http://my-api.fr/user/10', json.dumps(put_response))

# Récupération de la réponse depuis le cache
cached_response = r.get('http://my-api.fr/user/10')

# Affichage de la réponse
print(json.loads(cached_response))

{'id': 10, 'name': 'jean'}


**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [60]:
# Création de la requête
get_response = [
    {"id": 10, "name": "jean"},
    {"id": 11, "name": "Claire"}
]

# Insertion de la réponse dans le cache
r.set('http://my-api.fr/user?size=2', json.dumps(get_response))

# Récupération de la réponse depuis le cache
cached_response = r.get('http://my-api.fr/user?size=2')

# Affichage de la réponse
print(json.loads(cached_response))

[{'id': 10, 'name': 'jean'}, {'id': 11, 'name': 'Claire'}]


**Q** : Create a set of cache request keys.

In [61]:
# Création de la requête
cache_keys = [
    'http://my-api.fr/user/10',
    'http://my-api.fr/user?size=2',
    'http://my-api.fr/user/11'
]

# Ajout des clés dans un ensemble
r.sadd('cache_keys', *cache_keys)

# Récupération des clés du cache
all_cache_keys = r.smembers('cache_keys')

# Affichage des clés du cache
print(all_cache_keys)

{b'http://my-api.fr/user/10', b'http://my-api.fr/user/11', b'http://my-api.fr/user?size=2'}


**Q** : Check if the GET request on http: //http://my-api.fr/user

In [62]:
# Paramétrage de la clé à vérifier
key_to_check = 'http://my-api.fr/user'

# Vérification de la présence de la clé dans l'ensemble
is_member = r.sismember('cache_keys', key_to_check)

# Affichage du résultat
print(f"La clé '{key_to_check}' est dans le set 'cache_keys' : {is_member}")

La clé 'http://my-api.fr/user' est dans le set 'cache_keys' : 0


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [63]:
# Paramétrage de la clé à supprimer
deleted_count = r.delete('http://my-api.fr/user/10')

# Affichage du nombre de clés supprimées
print(f"Nombre de clés supprimées : {deleted_count}")

Nombre de clés supprimées : 1


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [64]:
!redis-cli flushall

'redis-cli' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
